In [21]:
#Model Train/Test (70/30) 
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
#from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
# Load merged dataset
df_dnn=pd.read_csv("road/preprocessed/merged/multiclass_attack_data_without_masquerade.csv")  
df=pd.read_csv("road/preprocessed/merged/multiclass_attack_data_without_masquerade.csv")


X_dnn = df_dnn.drop(columns=['Flag'], errors='ignore')
y_dnn = df_dnn['Flag']
X = df.drop(columns=['Flag'], errors='ignore')
y = df['Flag']

# Stratified split
X_train_dnn, X_test_dnn, y_train_dnn, y_test_dnn = train_test_split(
    X_dnn, y_dnn, train_size=0.7, random_state=42, stratify=y
)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, random_state=42, stratify=y
)
# Clean column names 
X_train = X_train.copy()
X_test = X_test.copy()
X_train.columns = [c.replace("[", "_").replace("]", "").replace("<", "_") for c in X_train.columns]
X_test.columns = [c.replace("[", "_").replace("]", "").replace("<", "_") for c in X_test.columns]

# Confirm sizes
print("Train samples:", len(X_train_dnn))
print("Test samples:", len(X_test_dnn))
print("Train samples:", len(X_train))
print("Test samples:", len(X_test))
results = []

Train samples: 1078308
Test samples: 462133
Train samples: 1078308
Test samples: 462133


In [13]:
display(df_dnn['Flag'].value_counts())
display(df['Flag'].value_counts())

Flag
0    1490778
3      17221
6      15195
5      10605
1       5493
2       1061
4         88
Name: count, dtype: int64

Flag
0    1490778
3      17221
6      15195
5      10605
1       5493
2       1061
4         88
Name: count, dtype: int64

In [ ]:
"""----------DNN----------"""
from tensorflow.keras.callbacks import EarlyStopping
# ----- One-hot encode labels -----
y_train_cat = to_categorical(y_train_dnn, num_classes=7)
y_test_cat = to_categorical(y_test_dnn, num_classes=7)

# Compute class weights for imbalance handling
#class_weights = compute_class_weight(class_weight='balanced',
                                   #  classes=np.unique(y_train),
                                   #  y=y_train)
#class_weights_dict = dict(enumerate(class_weights))

#batch_sizes = [256]
#histories = {}
#results=[]
#for bs in batch_sizes:
print(f"\nTraining with batch size: 32")
model = Sequential([   
Input(shape=(X_train_dnn.shape[1],)),  # Input layer (should be 10 features)
Dense(16, activation='relu'),#Dropout(0.3),
Dense(16, activation='relu'), #Dropout(0.3),
Dense(16, activation='relu'),#Dropout(0.3),
Dense(16, activation='relu'),#Dropout(0.3),
Dense(7, activation='softmax')  # Output layer: 5 neurons
])  # Rebuild model from scratch
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stop = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train_dnn, y_train_cat,
    validation_split=0.1,
    epochs=50,
    batch_size=32, 
    verbose=1,callbacks[early_stop]
)

# ----- Predict -----
y_pred_dnn_prob = model.predict(X_test_dnn, batch_size=1024)
y_pred_dnn = np.argmax(y_pred_dnn_prob, axis=1)

# ----- Evaluation -----
# Convert multiclass to binary: 0 = normal, 1 = any attack
y_test_binary = (y_test_dnn != 0).astype(int)
y_pred_dnn_binary = (y_pred_dnn != 0).astype(int)

# Calculate F1 score and confusion matrix
f1 = f1_score(y_test_binary, y_pred_dnn_binary, zero_division=0)
cm = confusion_matrix(y_test_binary, y_pred_dnn_binary, labels=[0, 1])
tn, fp, fn, tp = cm.ravel()

# Benign = all benign test samples = TN + FP
# Malicious = all malicious test samples = TP + FN
# Benign / Malicious counts (total in test set per class) 
benign_count = int((y_test_binary == 0).sum())     # = tn + fp
malicious_count = int((y_test_binary == 1).sum())  # = tp + fn

dnn=model
# Store results table 
results.append(["DNN", benign_count, malicious_count, f"{f1*100:.1f}%", fn, fp])
results_df = pd.DataFrame(results, 
    columns=["Model", "Benign Samples", "Malicious Samples", "F1 score", "FN", "FP"])
print(results_df)

#histories[bs] = history.history


Training with batch size: 32
Epoch 1/50
30328/30328 [==============================] - 95s 3ms/step - loss: 0.1012 - accuracy: 0.9796 - val_loss: 0.0525 - val_accuracy: 0.9857
Epoch 2/50
30328/30328 [==============================] - 91s 3ms/step - loss: 0.0435 - accuracy: 0.9869 - val_loss: 0.0399 - val_accuracy: 0.9875
Epoch 3/50
30328/30328 [==============================] - 94s 3ms/step - loss: 0.0406 - accuracy: 0.9874 - val_loss: 0.0400 - val_accuracy: 0.9885
Epoch 4/50
30328/30328 [==============================] - 92s 3ms/step - loss: 0.0401 - accuracy: 0.9875 - val_loss: 0.0370 - val_accuracy: 0.9885
Epoch 5/50
30328/30328 [==============================] - 92s 3ms/step - loss: 0.0415 - accuracy: 0.9875 - val_loss: 0.0379 - val_accuracy: 0.9883
Epoch 6/50
30328/30328 [==============================] - 98s 3ms/step - loss: 0.0395 - accuracy: 0.9875 - val_loss: 0.0364 - val_accuracy: 0.9882
Epoch 7/50
30328/30328 [==============================] - 93s 3ms/step - loss: 0.0393 - 

In [ ]:


# Models dict (loaded/trained models)
models = {
    "DT": DecisionTreeClassifier(random_state=42),
    "RF": RandomForestClassifier(n_estimators=100, random_state=42),
    "ET": ExtraTreesClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
}


# number of classes 
n_classes = len(np.unique(y_train))  # ensure this matches preprocessing

for name, model in models.items():
    # ----- Train (multiclass) -----
    model.fit(X_train, y_train)   # y_train contains 0 for normal, 1..K for attacks

    # ----- Predict multiclass -----
    y_pred = model.predict(X_test)   # multiclass predictions

    # ----- Collapse to binary: 0 = normal, 1 = any attack -----
    y_test_bin = (y_test != 0).astype(int)
    y_pred_bin = (y_pred != 0).astype(int)

    # ----- Compute metrics -----
    cm = confusion_matrix(y_test_bin, y_pred_bin, labels=[0, 1])
    tn, fp, fn, tp = cm.ravel()

    # Benign / Malicious counts (total in test set per class):
    benign_count = int((y_test_bin == 0).sum())     # = tn + fp
    malicious_count = int((y_test_bin == 1).sum())  # = tp + fn

    f1 = f1_score(y_test_bin, y_pred_bin, zero_division=0)

    results.append([
        name,
        benign_count,
        malicious_count,
        f"{f1*100:.1f}%",
        int(fn),
        int(fp)
    ])

# Build DataFrame 
results_df = pd.DataFrame(results, columns=[
    "Model", "Benign Samples", "Malicious Samples", "F1 score", "FN", "FP"
])

print(results_df)


c:\Users\Lenovo\anaconda3\envs\ivn-ids\lib\site-packages\xgboost\training.py:183: UserWarning: [12:56:02] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


     Model  Benign Samples  Malicious Samples F1 score    FN   FP
0      DNN          447234              14899    75.9%  5482  502
1       DT          447234              14899    90.9%  1680  956
2       RF          447234              14899    91.0%  1664  940
3       ET          447234              14899    91.0%  1692  925
4  XGBoost          447234              14899    91.0%  1653  954


In [16]:
dnn.save("road/models/dnn_model_multiclass.h5")

NameError: name 'dnn' is not defined

In [ ]:
import tensorflow as tf
print(tf.__version__)


Training with batch size: 32
Epoch 1/5
3791/3791 [==============================] - 46s 10ms/step - loss: -3729298.5000 - accuracy: 0.8530 - val_loss: -20114700.0000 - val_accuracy: 0.9016
Epoch 2/5
3791/3791 [==============================] - 14s 4ms/step - loss: -132324192.0000 - accuracy: 0.8578 - val_loss: -329324544.0000 - val_accuracy: 0.9009
Epoch 3/5
3791/3791 [==============================] - 12s 3ms/step - loss: -883374528.0000 - accuracy: 0.8570 - val_loss: -1626084096.0000 - val_accuracy: 0.8698
Epoch 4/5
3791/3791 [==============================] - 12s 3ms/step - loss: -3206748160.0000 - accuracy: 0.8547 - val_loss: -5063260672.0000 - val_accuracy: 0.8975
Epoch 5/5
452/452 [==============================] - 1s 3ms/step


,Model,Benign Samples,Malicious Samples,F1 score,FN,FP
0,DNN,411784,8289,28.3%,6610,35450
